In [2]:
from osgeo import gdal, gdalconst
import os
import numpy as np

In [1]:
"""For small rasters"""
def reclassify_raster(input_path, output_path, reclass_dict):
    # Open the input raster
    input_ds = gdal.Open(input_path, gdalconst.GA_ReadOnly)
    if input_ds is None:
        print(f"Error: Unable to open input raster {input_path}")
        return

    # Get raster information
    rows = input_ds.RasterYSize
    cols = input_ds.RasterXSize
    bands = input_ds.RasterCount

    # Create an output raster
    driver = gdal.GetDriverByName('GTiff')
    output_ds = driver.Create(output_path, cols, rows, bands, 
                              gdal.GDT_Byte, # CUIDADO AQUI
                              options=['COMPRESS=DEFLATE', 'TILED=YES'])
    output_ds.SetProjection(input_ds.GetProjection())
    output_ds.SetGeoTransform(input_ds.GetGeoTransform())

    # Reclassify each pixel
    for band in range(1, bands + 1):
        input_band = input_ds.GetRasterBand(band)
        output_band = output_ds.GetRasterBand(band)
        
        """ SET NODATA """
        output_band.SetNoDataValue(0)  # Set nodata value to 0 **CUIDADO**

        data = input_band.ReadAsArray()

        # Reclassify using the provided dictionary
        for key, value in reclass_dict.items():
            data[data == key] = value

        output_band.WriteArray(data)

    # Close datasets
    input_ds = None
    output_ds = None

In [3]:
"""If the layer is too big"""
def reclassify_raster(input_path, output_path, reclass_dict, block_size=512):
    # Open the input raster
    input_ds = gdal.Open(input_path, gdalconst.GA_ReadOnly)
    if input_ds is None:
        print(f"Error: Unable to open input raster {input_path}")
        return

    # Get raster information
    rows = input_ds.RasterYSize
    cols = input_ds.RasterXSize
    bands = input_ds.RasterCount

    # Create an output raster
    driver = gdal.GetDriverByName('GTiff')
    output_ds = driver.Create(output_path, cols, rows, bands, 
                              gdal.GDT_Byte,
                              options=['COMPRESS=DEFLATE', 'TILED=YES', 'COPY_SRC_OVERVIEWS=YES'])
    output_ds.SetProjection(input_ds.GetProjection())
    output_ds.SetGeoTransform(input_ds.GetGeoTransform())

    # Reclassify each pixel in blocks
    for band in range(1, bands + 1):
        input_band = input_ds.GetRasterBand(band)
        output_band = output_ds.GetRasterBand(band)

        # Set NODATA value
        output_band.SetNoDataValue(0)

        # Process the raster in blocks
        for i in range(0, rows, block_size):
            for j in range(0, cols, block_size):
                block_rows = min(block_size, rows - i)
                block_cols = min(block_size, cols - j)

                data = input_band.ReadAsArray(j, i, block_cols, block_rows)

                # Reclassify using the provided dictionary
                for key, value in reclass_dict.items():
                    if isinstance(key, tuple):  # Handle ranges
                        low, high = key
                        data[(data >= low) & (data <= high)] = value
                    else:
                        data[data == key] = value

                output_band.WriteArray(data, j, i)

    # Close datasets
    input_ds = None
    output_ds = None

In [4]:
if __name__ == "__main__":
    input_folder = r"C:\Users\admin\Downloads\global_forest_watch_data\test"
    output_folder = r"C:\Users\admin\Downloads\global_forest_watch_data\test\output"
    """Be careful in the order you write the reclass"""
    # reclass_dict = {
    #     111: 1, 112: 2, 121: 3, 122: 4, 123: 5, 124: 6,
    #     131: 7, 132: 8, 133: 9, 141: 10, 142: 11, 211: 12,
    #     212: 13, 213: 14, 221: 15, 222: 16, 223: 17, 231: 18,
    #     241: 19, 242: 20, 243: 21, 244: 22, 311: 23, 312: 24,
    #     313: 25, 321: 26, 322: 27, 323: 28, 324: 29, 331: 30,
    #     332: 31, 333: 32, 334: 33, 335: 34, 411: 35, 412: 36,
    #     421: 37, 422: 38, 423: 39, 511: 40, 512: 41, 521: 42,
    #     522: 43, 523: 44, 999: np.nan
    # }
    
    # reclass_dict = {
    #     101: 0, 102: 255
    # }

    # reclass_dict = {
    #     (1, 100): 2,
    #     0: 1,
    #     255: 0
    # }

    reclass_dict = {
        # 254: 255,
    }
    """direct values"""
    # reclass_dict = {
    #     np.nan: 0
    # }
    """ranges"""
    # reclass_dict = {
    #     (0, 100): 1,
    #     (101, 200): 2,
    #     (201, 255): 3
    # }

    # Loop through each raster in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".tif"):  # Assuming raster files are in GeoTIFF format
            input_path = os.path.join(input_folder, file_name)
            output_name = os.path.splitext(file_name)[0] + ".tif" #_reclass.tif
            output_path = os.path.join(output_folder, output_name)

            reclassify_raster(input_path, output_path, reclass_dict)
            print("finished: " + output_name)
    print("Reclassification complete.")

finished: treecover2010_20N_110e.tif
finished: treecover2010_20n_110w.tif
finished: treecover2010_20n_120e.tif
finished: treecover2010_20n_160w.tif
finished: treecover2010_20s_010e.tif
Reclassification complete.
